                                                 Домашнє завдання N 7

Так як ми працюємо з рекомендованними системами, по завданню нам радять використовувати бібліотеку Surprise, котра якраз є по суті додатком до знайомої нам бібліотеки scikit-learn для тренування моделей рекомендаційних систем.

Тому давайте імпортуємо цю бібліотеку і також завантажимо потрібний датасет movielens на 100 к даних. Думаю, нам буде достатньо цих даних для виконання домашнього завдання:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from surprise import Dataset, Reader, SVD, SVDpp, NMF, accuracy
from surprise.model_selection import cross_validate, GridSearchCV, train_test_split

In [22]:
dataset_path = './ml-100k/u.data'
reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_file(dataset_path, reader=reader)

data

Давайте спробуємо вивести перші 5 рядків, щоб побачити, які дані містить наш датасет. Для цього використаємо трохи інший ніж зазвичай метод виводу:

In [23]:
N = 5
data_raw = data.raw_ratings

for i in range(N):
    print(data_raw[i])



('196', '242', 3.0, '881250949')
('186', '302', 3.0, '891717742')
('22', '377', 1.0, '878887116')
('244', '51', 2.0, '880606923')
('166', '346', 1.0, '886397596')


Добре, супер. Для більш наглядності давайте виведемо для користувача id 420 та рейтинги, які він виставив:

In [62]:
dataset_path = './ml-100k/u.data'
ratings_df = pd.read_csv(dataset_path, sep='\t', names=['user_id', 'movie_id', 'rating', 'timestamp'])

user_420_ratings = ratings_df[ratings_df['user_id'] == 420]

# Виведемо рейтинги цього користувача
print(user_420_ratings[['user_id', 'movie_id', 'rating']])

       user_id  movie_id  rating
14687      420       331       3
15906      420        19       3
17750      420       283       5
19368      420       302       4
20639      420       137       4
23378      420       319       4
23879      420       753       5
24204      420       275       5
24306      420       251       5
28022      420       288       3
32760      420       475       4
32827      420       408       4
33697      420       116       4
36166      420       100       5
36585      420       508       3
40355      420       478       3
44659      420       603       4
51689      420       750       4
51774      420       513       5
51993      420        14       5
52090      420        86       5
53890      420       301       3
54382      420       484       5
54891      420       270       3
66828      420       173       3
69809      420      1347       3
71530      420       190       5
79095      420       124       5
79151      420       127       5
79254     

Окей, супер!

Тепер давайте побудуємо модель за допомогою 3 алгоритмів розграхунків: SVD, SVD++, NMF та вкористаємо cross_validation для пошуку параметрів RMSE та MAE

Почнемо з SVD:

In [4]:
algo = SVD()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9357  0.9305  0.9386  0.9349  0.9426  0.9364  0.0040  
MAE (testset)     0.7381  0.7338  0.7407  0.7381  0.7418  0.7385  0.0028  
Fit time          0.38    0.39    0.43    0.39    0.39    0.40    0.02    
Test time         0.04    0.08    0.04    0.07    0.04    0.05    0.02    


{'test_rmse': array([0.93569515, 0.93045161, 0.93857916, 0.93486193, 0.9425569 ]),
 'test_mae': array([0.73809328, 0.73378101, 0.7407054 , 0.73806817, 0.74181418]),
 'fit_time': (0.38176608085632324,
  0.39353394508361816,
  0.43080925941467285,
  0.3888368606567383,
  0.38868093490600586),
 'test_time': (0.03812980651855469,
  0.07621502876281738,
  0.0389561653137207,
  0.07130885124206543,
  0.03742218017578125)}

Тепер те саме зробимо з SVD++:

In [5]:
algo_svdpp = SVDpp()

cross_validate(algo_svdpp, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9194  0.9212  0.9150  0.9241  0.9169  0.9193  0.0032  
MAE (testset)     0.7209  0.7225  0.7177  0.7241  0.7195  0.7210  0.0022  
Fit time          6.14    6.14    6.15    6.12    6.15    6.14    0.01    
Test time         1.29    1.25    1.29    1.26    1.25    1.27    0.02    


{'test_rmse': array([0.91937674, 0.92115591, 0.91502694, 0.92406008, 0.91691509]),
 'test_mae': array([0.7209369 , 0.72249581, 0.71774912, 0.72408846, 0.71954958]),
 'fit_time': (6.138238906860352,
  6.142917156219482,
  6.145297050476074,
  6.117411851882935,
  6.150583982467651),
 'test_time': (1.2905406951904297,
  1.2522492408752441,
  1.2868480682373047,
  1.25888991355896,
  1.2521281242370605)}

І також використовуючи NMF модель:

In [6]:
algo_nmf = NMF()

cross_validate(algo_nmf, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9638  0.9603  0.9617  0.9602  0.9630  0.9618  0.0014  
MAE (testset)     0.7566  0.7535  0.7557  0.7546  0.7577  0.7556  0.0015  
Fit time          0.50    0.54    0.54    0.54    0.50    0.53    0.02    
Test time         0.07    0.03    0.07    0.03    0.07    0.05    0.02    


{'test_rmse': array([0.96375757, 0.96030764, 0.96173788, 0.96023495, 0.96297855]),
 'test_mae': array([0.75660726, 0.7535099 , 0.75568642, 0.75463535, 0.75773559]),
 'fit_time': (0.4970591068267822,
  0.54498291015625,
  0.5447478294372559,
  0.5428762435913086,
  0.5001728534698486),
 'test_time': (0.06875205039978027,
  0.03265094757080078,
  0.07032227516174316,
  0.03178691864013672,
  0.07007312774658203)}

Ми отримали результати оцінок RMSE та MAE всіх 3 алгоритмів. Давайте проаналізуємо отримані дані та зробимо висновки:

Алгоритм SVD: Має середнє значення RMSE приблизно 0.9345 і MAE приблизно 0.7373. Цей алгоритм показав непогані результати з помірним часом навчання та тестування.

Алгоритм SVD++: Показав кращі результати за RMSE (середнє 0.9194) та MAE (середнє 0.7213) порівняно з SVD, що свідчить про вищу точність прогнозування. Однак, час навчання та тестування значно вищий, що є важливим аспектом для великих наборів даних або обмежених обчислювальних ресурсів.

Алгоритм NMF: Має найвищі значення помилок RMSE (середнє 0.9648) та MAE (середнє 0.7572) серед трьох розглянутих алгоритмів, що вказує на нижчу точність прогнозування порівняно з SVD та SVD++. Час навчання та тестування подібний до SVD.

Як висновок, можемо зазначити, що найбільш вдалим тут здається вибір алгоритму SVD++, так як в нас датасет не такий великий. Якби ми взяли датасет не на 100к а на 1М даних, то досить велика ймовірність, що краще б було тоді застосовувати алгоритм SVD.

Ми знайшли оптимальний алгоритм на основі пошуку параметрів RMSE,MAE через cross_validation.

Тепер же давайте застосуємо метод GridSearchCV для пошуку оптимальних гіперпараметрів. Для цього використаємо розділ документації бібліотеки surprise: Tune algorithm parameters with GridSearchCV.

Зробимо це для алгоритму SVD++, так як він показався найбільш оптимальним, хоча достатньо ресурсно-залежний:

In [7]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVDpp, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for SVD++: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for SVD++: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_svd_pp = SVDpp(**gs.best_params['rmse'])
algo_mae_svd_pp = SVDpp(**gs.best_params['mae'])

RMSE for SVD++: 0.9640980905865284
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE for SVD++: 0.7722318559600271
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


Тепер, коли у нас є модель, яка оптимізована за обома метриками і підібрані гіперпараметри, ми можемо провести оцінку на тестовому наборі для перевірки точності. Для цього розділимо наш датасет на тренувальний і тестовий у пропорції 80 на 20:

In [8]:
trainset, testset = train_test_split(data, test_size=0.2)

Тепер навчимо нашу модель на тренувальному наборі:

In [9]:
algo_rmse_svd_pp.fit(trainset)
algo_mae_svd_pp.fit(trainset)

Тепер отримаємо прогнози на нашому тестовому наборі:

In [10]:
predictions_rmse = algo_rmse_svd_pp.test(testset)
predictions_mae = algo_mae_svd_pp.test(testset)

І тепер проведемо оцінку моделі:

In [11]:
accuracy_rmse_svd_pp = accuracy.rmse(predictions_rmse)
accuracy_mae_svd_pp = accuracy.mae(predictions_mae)

RMSE: 0.9607
MAE:  0.7703


Давайте зробимо пошук оптимальних гіперпараметрів для алгоритмів SVD та NMF і потім оцінимо отримані результати.

Для алгоритму SVD:

In [12]:
param_grid = {"n_epochs": [5, 10], "lr_all": [0.002, 0.005], "reg_all": [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for SVD: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for SVD: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_svd = SVD(**gs.best_params['rmse'])
algo_mae_svd = SVD(**gs.best_params['mae'])

RMSE for SVD: 0.9637752587551272
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}
MAE for SVD: 0.772306618348089
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


In [13]:
trainset, testset = train_test_split(data, test_size=0.2)

algo_rmse_svd.fit(trainset)
algo_mae_svd.fit(trainset)

predictions_rmse = algo_rmse_svd.test(testset)
predictions_mae = algo_mae_svd.test(testset)

accuracy_rmse_svd = accuracy.rmse(predictions_rmse)
accuracy_mae_svd = accuracy.mae(predictions_mae)

RMSE: 0.9647
MAE:  0.7746


Для алгоритму NMF:

In [14]:
param_grid = {"n_epochs": [5, 10], "n_factors": [15, 50, 100], "reg_pu": [0.05, 0.1, 0.15], "reg_qi": [0.05, 0.1, 0.15]}
gs = GridSearchCV(NMF, param_grid, measures=["rmse", "mae"], cv=3)

gs.fit(data)

print(f"RMSE for NMF: {gs.best_score['rmse']}")
print(gs.best_params['rmse'])
print(f"MAE for NMF: {gs.best_score['mae']}")
print(gs.best_params['mae'])

algo_rmse_nmf = NMF(**gs.best_params['rmse'])
algo_mae_nmf = NMF(**gs.best_params['mae'])

RMSE for NMF: 0.9679040389311085
{'n_epochs': 10, 'n_factors': 15, 'reg_pu': 0.15, 'reg_qi': 0.15}
MAE for NMF: 0.7614099271082763
{'n_epochs': 10, 'n_factors': 15, 'reg_pu': 0.15, 'reg_qi': 0.1}


In [15]:
trainset, testset = train_test_split(data, test_size=0.2)

algo_rmse_nmf.fit(trainset)
algo_mae_nmf.fit(trainset)

predictions_rmse = algo_rmse_nmf.test(testset)
predictions_mae = algo_mae_nmf.test(testset)

accuracy_rmse_nmf = accuracy.rmse(predictions_rmse)
accuracy_mae_nmf = accuracy.mae(predictions_mae)

RMSE: 0.9692
MAE:  0.7639


In [18]:
print(f"SVD: RMSE - {accuracy_rmse_svd}, MAE - {accuracy_mae_svd}")
print(f"SVD++: RMSE - {accuracy_rmse_svd_pp}, MAE - {accuracy_mae_svd_pp}")
print(f"NMF: RMSE - {accuracy_rmse_nmf}, MAE - {accuracy_mae_nmf}")


SVD: RMSE - 0.9646623069875646, MAE - 0.7745653492000579
SVD++: RMSE - 0.9607294767829024, MAE - 0.7703349619889711
NMF: RMSE - 0.9691783008863051, MAE - 0.7639054050702421


Здається, алгоритм SVD++ показав найкращий результат за метрикою RMSE (0.9607) та майже найкращий за MAE (0.7703), що свідчить про добру здатність моделі загалом передбачати оцінки, особливо з точки зору квадратичних помилок. Але це можливо повʼязано з правильним підбором параметрів в param_grid.

По методу cross_validation найкращим також був SVD++. 
Можливо, це повʼязано з тим, що в param_grid для GridSearchCV я задав не найкращі параметри для SVD++...

Тепер, коли модель навчена, ми можемо використовувати її для генерації рекомендацій для конкретного користувача за допомогою SVD++.

Для цього давайте роділимо наш датасет на тренувальну та тестову вибірку і зробимо тренування моделі SVD++:

In [53]:
trainset = data.build_full_trainset()

algo = SVDpp()
algo.fit(trainset)

Тепер завантажимо список усіх фільмів та ідентифікатор користувача.. Наприклад, для користувача id 420:

In [54]:
items = trainset.all_items()
item_ids = [trainset.to_raw_iid(item) for item in items]

user_id = 420
predictions = [algo.predict(user_id, item_id) for item_id in item_ids]

Далі проведемо сортування прогнозів за оцінками:

In [55]:
predictions.sort(key=lambda x: x.est, reverse=True)

Завантажимо датафрейм метаданих фільмів:

In [56]:
movie_metadata_df = pd.read_csv('./ml-100k/u.item', sep='|', encoding='ISO-8859-1', names=['movie_id', 'title'], usecols=[0, 1])
movie_metadata_df

,movie_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)
...,...,...
1677,1678,Mat' i syn (1997)
1678,1679,B. Monkey (1998)
1679,1680,Sliding Doors (1998)
1680,1681,You So Crazy (1994)


Далі вже вибираємо кількість рекомендацій, яку хочемо задати для відтворення і робимо фільтрацію датафрейму фільмів і відповідно виводимо рекомендації для користувача.. 

В нашому випадку, зробимо топ 10 рекомендацій для користувача під id 420:

In [57]:
top_n = 10
recommended_movie_ids = [pred.iid for pred in predictions[:top_n]]
recommended_movie_ids_int = [int(iid) for iid in recommended_movie_ids]


recommended_movies_corrected = movie_metadata_df[movie_metadata_df['movie_id'].isin(recommended_movie_ids_int)]

print(recommended_movies_corrected)

      movie_id                                              title
11          12                         Usual Suspects, The (1995)
63          64                   Shawshank Redemption, The (1994)
113        114  Wallace & Gromit: The Best of Aardman Animatio...
168        169                         Wrong Trousers, The (1993)
317        318                            Schindler's List (1993)
407        408                              Close Shave, A (1995)
482        483                                  Casablanca (1942)
512        513                              Third Man, The (1949)
602        603                                 Rear Window (1954)
1448      1449                             Pather Panchali (1955)


Здається, ми успішно реалізували виведення топ 10 рекомендованих фільмів для користувача з id 420.